<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/006_Visualiza%C3%A7%C3%A3o_%22embeddings%22_da_CNN1D_modelo_treinado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import os
import pandas as pd

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Input, Embedding, Activation, Flatten, Dense
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from mpl_toolkits import mplot3d
import plotly.express as px

In [2]:
model = tf.keras.models.load_model("CNN1D_DESC_NCMt.H5", compile=False)

In [3]:
#Copia os dados das NFEs do portal da cidadância pro drive virtual.
import requests  

DATA_FILE = "AFRAC-20220506.zip"
df = pd.read_csv(DATA_FILE, encoding='utf-8-sig',sep=";", decimal=',')
#(csv_file_name,encoding='utf-8-sig', sep =';', decimal=',', index=False)
print("O dataframe completo contém:",len(df),"linhas")

df3 = df[['Data de Emissão - Documento Fiscal','Descrição do Item', 'NCM', 'GTIN/EAN Comercial']]
df3.columns = ["DATA", "DESCR", "NCM", "GTIN"]
#df3 = df3[df3["NCM"]!=-1] #Filtrando NCMs = -1

text_lengths = np.array([len(_) for _ in df3['DESCR']])
df3 = df3[text_lengths>=3]

#Caso queira eliminar as repetições rodar a linha abaixo
df3 = df3.drop_duplicates(subset=["DESCR", "NCM", "GTIN"])
print("O dataframe sem repetiçoes contém:",len(df3),"linhas")
df3[:1]

O dataframe completo contém: 1363510 linhas
O dataframe sem repetiçoes contém: 1105737 linhas


,DATA,DESCR,NCM,GTIN
0,2022-03-05,!!!!! AGUA COM GAS 500ML,22021000,-1


In [4]:
descs = [_.lower() for _ in df3["DESCR"]]
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(descs)
print(tk.word_index)
print(f"{len(tk.word_index)} caracteres.")
print("".join(sorted([_ for _ in tk.word_index])))
descs = [_+(120-len(_))*" " for _ in descs]
df3["DESCR_NORM"] = descs
df3[-5:]

{'UNK': 1, ' ': 2, 'a': 3, 'o': 4, 'e': 5, 'r': 6, 'i': 7, 'c': 8, 'l': 9, 's': 10, 't': 11, 'n': 12, 'm': 13, '0': 14, 'p': 15, 'g': 16, 'd': 17, 'u': 18, 'b': 19, '1': 20, 'f': 21, 'h': 22, '2': 23, '5': 24, 'v': 25, '3': 26, 'k': 27, '4': 28, '.': 29, 'x': 30, '6': 31, '8': 32, '-': 33, '7': 34, '9': 35, 'j': 36, 'z': 37, '/': 38, 'y': 39, 'q': 40, 'w': 41, ',': 42, ':': 43, '(': 44, ')': 45, '_': 46, '#': 47, '+': 48, '*': 49, '%': 50, '"': 51, "'": 52, 'ç': 53, '&': 54, 'ã': 55, ';': 56, ']': 57, '[': 58, '$': 59, 'á': 60, '\\': 61, '|': 62, 'é': 63, 'º': 64, 'ó': 65, '!': 66, '=': 67, 'í': 68, 'ê': 69, '?': 70, '>': 71, '@': 72, '¿': 73, '°': 74, 'ú': 75, '\xa0': 76, 'ô': 77, 'â': 78, '`': 79, 'à': 80, 'õ': 81, 'ï': 82, 'ª': 83, '²': 84, '{': 85, '<': 86, '~': 87, 'è': 88, '§': 89, 'ø': 90, 'ñ': 91, '³': 92, 'û': 93, 'ù': 94, '\xad': 95, '}': 96, '\x81': 97, 'ä': 98, 'ü': 99, '¶': 100, '^': 101, '€': 102, '¹': 103, 'µ': 104, '®': 105, '¡': 106}
106 caracteres.
 !"#$%&'()*+,-./012

,DATA,DESCR,NCM,GTIN,DESCR_NORM
1363505,2022-03-05,ÓLEO DE SEMENTE DE ABÓBORA CAP. 80/1000mg - GL...,21069030,7898568353873,óleo de semente de abóbora cap. 80/1000mg - gl...
1363506,2022-03-05,Óleo de Soja Liza 900Ml,15079011,7896036090244,óleo de soja liza 900ml ...
1363507,2022-03-05,Óleo Essencial Hortelã BioEssência - 10ml,33012990,7898937573192,óleo essencial hortelã bioessência - 10ml ...
1363508,2022-03-05,Óleo Essencial Lavanda Francesa BioEssência - ...,33012990,7898937573246,óleo essencial lavanda francesa bioessência - ...
1363509,2022-03-05,Óleo Motor 20w50 Havoline,27101932,-1,óleo motor 20w50 havoline ...


In [5]:
samples = list(df3["DESCR_NORM"].sample(1000))

In [6]:
# Convert string to index
X_sequences = tk.texts_to_sequences(samples)
X = np.array(X_sequences, dtype='float32')
X.shape

(1000, 120)

In [11]:
layer = model.layers[-3]
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=layer.output)
embeds = intermediate_layer_model.predict(X)
embeds.shape

(1000, 128)

In [12]:
#from sklearn.decomposition import PCA
#pca = PCA(3)
#xform = pca.fit_transform(embeds)

In [13]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=0)
xform = tsne.fit_transform(embeds)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [14]:
table = [ [n,*c] for n, c in zip(samples,list(xform)) ]
df = pd.DataFrame(table, columns=["Name", "X", "Y", "Z"])
for col in ["X","Y","Z"]:
  df[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())
df[:3]

,Name,X,Y,Z
0,torsilax c/30 cpr leve + por - (-) ...,0.572900,0.368092,0.560978
1,chiclete un ...,0.626032,0.636099,0.473368
2,madelesma metaldeido 50g ...,0.570478,0.614583,0.375979


In [16]:
plt.figure(figsize=(20,20))
fig = px.scatter_3d(df.sample(50), x="X", y="Y", z="Z", opacity=0.5, text="Name")
fig.update_layout(
    title="DOC EMBEDDINGS IN 3D",
    legend_title="Descrições mais próximas",
    font=dict(
        size=12,
    ))
fig.show()

<Figure size 1440x1440 with 0 Axes>

In [17]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
xform2D = tsne.fit_transform(embeds)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [18]:
table = [ [n,*c] for n, c in zip(samples,list(xform2D)) ]
df2 = pd.DataFrame(table, columns=["Name", "X", "Y"])
for col in ["X","Y"]:
  df2[col] = (df2[col]-df2[col].min())/(df2[col].max()-df2[col].min())
df2

,Name,X,Y
0,torsilax c/30 cpr leve + por - (-) ...,0.968159,0.577149
1,chiclete un ...,0.618744,0.492475
2,madelesma metaldeido 50g ...,0.545244,0.592417
3,aquarela 12 cores 12ml 8880 c12 staedtler cx 1...,0.247976,0.507205
4,top travete bojo removivel nadador:cinza mescl...,0.086410,0.930687
...,...,...,...
995,papel hig folhalev neutro 8x30mt ...,0.477626,0.492604
996,lamina de serra 3.25 ...,0.436840,0.523050
997,brinco prata trio ...,0.215046,0.828933
998,fini beijo de morango 100g ...,0.609424,0.510202


In [19]:
fig = px.scatter(df.sample(50), x="X", y="Y", opacity=1, text="Name", size=[100]*50)

fig.show()